In [12]:
import sys, os, time
import numpy as np
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%matplotlib tk
import matplotlib.pyplot as plt

import tnt_util as util
from tnt_util import adict, idict, xset, collate, load, render_dict, get_object, save, Logger, seq_iterate
from tnt_setup import init_gamestate, setup_phase
from tnt_cards import load_card_decks, draw_cards
from collections import namedtuple
import random
from itertools import chain, product
from tnt_units import load_unit_rules
import tnt_setup as setup

from flask_app import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
ping()

'Backend active: use "init" to init game'

In [14]:
player = 'Axis'

In [15]:
out = format_msg_to_python(init_game(debug=False, player=player))

Beginning phase: Setup


In [16]:
player = 'West'

In [17]:
out = format_msg_to_python(get_status(player))

In [18]:
render_dict(out)

In [19]:
# action = random.choice(list(util.decode_actions(out.actions)))
# action

In [20]:
#out = take_action(player, action)

In [21]:
#out = format_msg_to_python(take_action(player, ('Britain', 'Paris', 'Infantry')))

In [22]:
for player in ['Axis', 'West', 'USSR']:
    out = format_msg_to_python(get_status(player))
    for _ in range(25):
        if 'error' in out:
            print(out.error)
            break
        elif 'actions' not in out:
            #render_dict(out)
            print('Player {} is waiting'.format(player))
            break
        action = random.choice(list(util.decode_actions(out.actions)))
        print(action)
        out = format_msg_to_python(take_action(player, action))

('Germany', 'Konigsberg', 'Fortress')
('Italy', 'Rome', 'Tank')
('Italy', 'Rome', 'AirForce')
('Italy', 'Rome', 'Fortress')
('Germany', 'Ruhr', 'Fleet')
('Germany', 'Konigsberg', 'Submarine')
('Italy', 'Rome', 'Fleet')
('Italy', 'Milan', 'Infantry')
('Italy', 'Milan', 'Fortress')
('Germany', 'Ruhr', 'Infantry')
('Germany', 'Ruhr', 'Tank')
('Germany', 'Munich', 'AirForce')
('Germany', 'Munich', 'Tank')
('Italy', 'Tripoli', 'Submarine')
('Germany', 'Berlin', 'Carrier')
('Italy', 'Tripoli', 'Carrier')
('Germany', 'Ruhr', 'Submarine')
('Germany', 'Berlin', 'Infantry')
('Germany', 'Berlin', 'Tank')
('Germany', 'Berlin', 'Carrier')
('Germany', 'Berlin', 'AirForce')
('Germany', 'Berlin', 'Infantry')
Player Axis is waiting
('Britain', 'Glasgow', 'Submarine')
('France', 'Marseille', 'Fortress')
('Britain', 'Suez', 'Submarine')
('Britain', 'London', 'Carrier')
('Britain', 'Bombay', 'Infantry')
('France', 'Algiers', 'Carrier')
('Britain', 'London', 'AirForce')
('France', 'Paris', 'Submarine')
('B

In [12]:
G = get_G()
render_dict(G)

In [10]:
# W = get_waiting_actions()
# render_dict(W)

In [11]:
# for faction, nationality, tile in power_iterate(G.temp.setup, None, 'cadres', None):
#     print(faction, nationality, tile)

In [2]:
a = adict()
load_unit_rules(a)
render_dict(a)

In [63]:
def power_iterate(content, *itrs):
    if len(itrs) == 0:
        try:
            yield from content
        except TypeError:
            yield content
        #yield content
    else:

        i, *itrs = itrs

        if isinstance(content, (list, tuple, set)):
            
            if i is None:
                for x in content:
                    yield from power_iterate(x, *itrs)
            elif isinstance(i, int):
                yield from power_iterate(content[i], *itrs)

            

        elif isinstance(content, dict):
            
            if i is None:
                for k, v in content.items():  # expand with id
                    for rest in power_iterate(v, *itrs):
                        if isinstance(rest, tuple):
                            yield (k,) + rest
                        else:
                            yield k, rest
            elif i in content:
                yield from power_iterate(content[i], *itrs)

                

In [67]:
for i, o in enumerate(power_iterate(a, 'units', None, None, 'A')):
    print(i, '\t', o)

0 	 ('rules', 'Fortress', 2)
1 	 ('rules', 'AirForce', 3)
2 	 ('rules', 'Carrier', 2)
3 	 ('rules', 'Submarine', 0)
4 	 ('rules', 'Fleet', 1)
5 	 ('rules', 'Tank', 0)
6 	 ('rules', 'Infantry', 1)
7 	 ('rules', 'Convoy', 0)
